# Desafio Técnico - Analista de Inteligência Artificial
# Análise de Vídeo, Classificação de Objetos e Processamento de Áudio


### Este notebook implementa uma solução completa para análise de cena gravada em vídeo

## Importação das Libs

- #### Separação do vídeo e Aúdio

- #### Extração de frames e armazenamento em banco

In [ ]:
# arquivos
import os

# Dados
import pandas as pd
import numpy as np
import sqlite3

# Imagens
import cv2
import pickle

from moviepy import VideoFileClip

# Plotagem
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE

# Utilitarios
import json

In [ ]:
caminho_arquivo_mp4 = "data/loveletter_bananagrams_caneca.mp4"

video_saida = "data/video_sem_audio.mp4"
audio_saida = "data/audio.mp3"

In [ ]:
if not os.path.exists(caminho_arquivo_mp4):
  print("Arquivo não encontrado")

In [ ]:
video = VideoFileClip(caminho_arquivo_mp4)

# 1. Separação do vídeo sem o aúdio

In [ ]:
video_sem_audio = video.without_audio()
video_sem_audio.write_videofile(video_saida, logger=None)

### Separação do aúdio

In [ ]:
audio = video.audio
audio.write_audiofile(audio_saida, logger=None)

### Fechamento dos recursos

In [ ]:
video.close()
audio.close()
video_sem_audio.close()

# 2.  Armazenamento e Tratamento de Imagens

#### Extração de frames

#### Criação de banco de dados SQLITE local

In [ ]:
db_folder = "data/db_sqlite"  # Pasta
db_file = "frames.db"          # Arquivo do banco
db_path = os.path.join(db_folder, db_file)

#### Criando tabela

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
                CREATE TABLE IF NOT EXISTS video_frames
                (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    video_name TEXT NOT NULL,
                    frame_number INTEGER NOT NULL,
                    timestamp REAL,
                    frame_data BLOB NOT NULL,
                    width INTEGER,
                    height INTEGER,
                    channels INTEGER DEFAULT 3,
                    color_space TEXT DEFAULT 'BGR',
                    mean_r REAL,
                    mean_g REAL,
                    mean_b REAL,
                    std_r REAL,
                    std_g REAL,
                    std_b REAL,
                    compression_type TEXT DEFAULT 'pickle',
                    custom_metadata TEXT,
                    scene_type TEXT,
                    motion_level TEXT,
                    lighting_condition TEXT,
                    notes TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    UNIQUE (frame_number)
                    )
                    ''')
conn.close()

#### Criando indices

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
            CREATE INDEX IF NOT EXISTS idx_video_name ON video_frames(video_name)
        ''')

cursor.execute('''
            CREATE INDEX IF NOT EXISTS idx_frame_number ON video_frames(frame_number)
        ''')

conn.commit()

conn.close()

#### Funções auxiliares

In [ ]:
def calculate_color_statistics(frame):
    """Calcula estatísticas de cor do frame"""
    if len(frame.shape) == 3:
        b, g, r = cv2.split(frame)
        stats = {
            'mean_r': float(np.mean(r)),
            'mean_g': float(np.mean(g)),
            'mean_b': float(np.mean(b)),
            'std_r': float(np.std(r)),
            'std_g': float(np.std(g)),
            'std_b': float(np.std(b))
        }
    else:
        # Frame em escala de cinza
        mean_val = float(np.mean(frame))
        std_val = float(np.std(frame))
        stats = {
            'mean_r': mean_val,
            'mean_g': mean_val,
            'mean_b': mean_val,
            'std_r': std_val,
            'std_g': std_val,
            'std_b': std_val
        }
    return stats

In [ ]:
def save_frame(video_name, frame_number, frame, timestamp=None, compression='pickle', metadata=None):
    """Salva um frame individual no banco de dados com análise de cor e metadados customizados"""
    # Calcula estatísticas do frame
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    stats = calculate_color_statistics(frame)

    # Processa metadados customizados
    custom_metadata_json = None
    scene_type = None
    motion_level = None
    lighting_condition = None
    notes = None

    if metadata:
        # Se metadata for um dicionário, converte para JSON
        if isinstance(metadata, dict):
            custom_metadata_json = json.dumps(metadata)
            # Extrai campos específicos se existirem
            scene_type = metadata.get('scene_type')
            motion_level = metadata.get('motion_level')
            lighting_condition = metadata.get('lighting_condition')
            notes = metadata.get('notes')
        else:
            # Se for string, usa como notes
            notes = str(metadata)

    # Converte o frame para bytes
    if compression == 'pickle':
        frame_bytes = pickle.dumps(frame)

    height, width = frame.shape[:2]
    channels = frame.shape[2] if len(frame.shape) == 3 else 1
    color_space = 'BGR' if channels == 3 else 'GRAY'

    cursor.execute('''
        INSERT OR REPLACE INTO video_frames
        (video_name, frame_number, timestamp, frame_data, width, height,
         channels, color_space, mean_r, mean_g, mean_b, std_r, std_g, std_b,
         compression_type, custom_metadata, scene_type,
         motion_level, lighting_condition, notes)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (video_name, frame_number, timestamp, frame_bytes, width, height,
              channels, color_space, stats['mean_r'], stats['mean_g'], stats['mean_b'],
              stats['std_r'], stats['std_g'], stats['std_b'], compression,
              custom_metadata_json, scene_type, motion_level, lighting_condition,
              notes))

    conn.commit()
    conn.close()

In [ ]:
def analyze_frame_content(frame, timestamp):
    """Exemplo de função para gerar metadados automaticamente"""
    height, width = frame.shape[:2]

    # Análise simples de brilho
    brightness = np.mean(frame)
    if brightness < 80:
        lighting = "dark"
    elif brightness > 180:
        lighting = "bright"
    else:
        lighting = "normal"

    # Análise de movimento (baseada em variação de pixels)
    frame_std = np.std(frame)
    if frame_std < 20:
        motion = "static"
    elif frame_std < 50:
        motion = "low"
    else:
        motion = "high"

    # Análise de cena (baseada em canais de cor)
    b, g, r = cv2.split(frame)
    avg_b, avg_g, avg_r = np.mean(b), np.mean(g), np.mean(r)

    if avg_g > avg_r and avg_g > avg_b:
        scene = "outdoor"  # Muito verde pode indicar vegetação
    elif avg_b > avg_r and avg_b > avg_g:
        scene = "sky"  # Muito azul pode ser céu
    else:
        scene = "indoor"  # Caso geral

    return {
        'scene_type': scene,
        'motion_level': motion,
        'lighting_condition': lighting,
        'brightness_value': float(brightness),
        'frame_std': float(frame_std),
        'resolution': f"{width}x{height}",
        'notes': f"Frame capturado em {timestamp:.1f}s"
    }

In [ ]:
cap = cv2.VideoCapture(video_saida)

if not cap.isOpened():
  raise ValueError(f"Erro ao abrir o vídeo: {video_saida}")

fps = cap.get(cv2.CAP_PROP_FPS)

time_interval = 0.1

frame_interval = int(fps * time_interval)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps
saved_count = 0
sample_index = 0
compression='pickle'

In [ ]:
while True:
    current_time = sample_index * time_interval

    if current_time >= duration:
        break

    target_frame = int(current_time * fps)

    cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

    ret, frame = cap.read()
    if not ret:
        break

    metadata = analyze_frame_content(frame, current_time)

    rounded_timestamp = round(current_time, 2)
    save_frame('video_sem_audio', target_frame, frame, rounded_timestamp, compression, metadata)
    saved_count += 1

    if saved_count % 10 == 0:
        progress = (current_time / duration) * 100
        print(f"Progresso: {progress:.1f}% - Tempo: {current_time:.1f}s - Frames salvos: {saved_count}")

    sample_index += 1

cap.release()

# Análise e Processamento de Imagens

#### Funções auxiliares

In [ ]:
def calculate_histogram(frame):
    """Calcula histograma da imagem"""
    histograms = {}

    if len(frame.shape) == 2:  # Imagem em escala de cinza
        histograms['gray'] = cv2.calcHist([frame], [0], None, [256], [0, 256]).flatten()
    else:  # Imagem colorida (BGR)
        colors = ['blue', 'green', 'red']
        for i, color in enumerate(colors):
            histograms[color] = cv2.calcHist([frame], [i], None, [256], [0, 256]).flatten()

    return histograms

In [ ]:
def load_frame_from_db(frame_id):
    """Carrega um frame específico do banco de dados"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute('''
        SELECT frame_data, compression_type FROM video_frames WHERE id = ?
    ''', (frame_id,))

    result = cursor.fetchone()
    if not result:
        return None

    frame_bytes, compression = result
    frame = pickle.loads(frame_bytes)

    conn.close()
    return frame

In [ ]:
def plot_histogram(frame, title="Histograma", save_path=None):
    """Plota o histograma de uma imagem"""
    histograms = calculate_histogram(frame)

    plt.figure(figsize=(10, 6))

    if 'gray' in histograms:
        # Imagem em escala de cinza
        plt.plot(histograms['gray'], color='black', alpha=0.7)
        plt.title(f'{title} - Escala de Cinza')
        plt.xlabel('Intensidade do Pixel')
        plt.ylabel('Frequência')
    else:
        # Imagem colorida
        colors = ['blue', 'green', 'red']
        for color in colors:
            plt.plot(histograms[color], color=color, alpha=0.7, label=color.capitalize())

        plt.title(f'{title} - RGB')
        plt.xlabel('Intensidade do Pixel')
        plt.ylabel('Frequência')
        plt.legend()

    plt.xlim([0, 256])
    plt.grid(True, alpha=0.3)

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Histograma salvo em: {save_path}")

    plt.close()

In [ ]:
def apply_clahe(self, frame, clip_limit=2.0, tile_grid_size=(8,8)):
    """Aplica CLAHE (Contrast Limited Adaptive Histogram Equalization)"""
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    if len(frame.shape) == 3:
        # Imagem colorida - aplica CLAHE no canal Y (YUV)
        yuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
        yuv[:,:,0] = clahe.apply(yuv[:,:,0])
        result = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    else:
        # Imagem em escala de cinza
        result = clahe.apply(frame)

    return result

In [ ]:
def equalize_histogram(frame):
    """Aplica equalização de histograma na imagem"""
    if len(frame.shape) == 3:
        # Imagem colorida - converte para YUV, equaliza Y, converte de volta
        yuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
        yuv[:,:,0] = cv2.equalizeHist(yuv[:,:,0])
        equalized = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    else:
        # Imagem em escala de cinza
        equalized = cv2.equalizeHist(frame)

    return equalized

In [ ]:
def compare_histograms(frame_original, frame_equalized, title="Comparação", save_path=None):
    """Compara histogramas antes e depois da equalização"""
    hist_original = calculate_histogram(frame_original)
    hist_equalized = calculate_histogram(frame_equalized)

    if 'gray' in hist_original:
        # Escala de cinza
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

        ax1.plot(hist_original['gray'], color='black', alpha=0.7)
        ax1.set_title('Histograma Original')
        ax1.set_xlabel('Intensidade')
        ax1.set_ylabel('Frequência')
        ax1.grid(True, alpha=0.3)

        ax2.plot(hist_equalized['gray'], color='black', alpha=0.7)
        ax2.set_title('Histograma Equalizado')
        ax2.set_xlabel('Intensidade')
        ax2.set_ylabel('Frequência')
        ax2.grid(True, alpha=0.3)
    else:
        # Colorida
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        colors = ['blue', 'green', 'red']

        for color in colors:
            ax1.plot(hist_original[color], color=color, alpha=0.7, label=color.capitalize())
            ax2.plot(hist_equalized[color], color=color, alpha=0.7, label=color.capitalize())

        ax1.set_title('Histograma Original')
        ax1.set_xlabel('Intensidade')
        ax1.set_ylabel('Frequência')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        ax2.set_title('Histograma Equalizado')
        ax2.set_xlabel('Intensidade')
        ax2.set_ylabel('Frequência')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

    plt.suptitle(title)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Comparação salva em: {save_path}")

    # IMPORTANTE: Fecha a figura para liberar memória
    plt.close()

In [ ]:
output_histogram_analysis = "data/histogram_analysis"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute(f'''
                SELECT id, video_name, frame_number, timestamp, compression_type
                FROM video_frames
                ORDER BY frame_number
            ''')

frames_info = cursor.fetchall()
conn.close()

if not frames_info:
    print("Nenhum frame encontrado no banco de dados!")

In [ ]:
use_clahe = False

for i, (frame_id, vid_name, frame_num, timestamp, compression) in enumerate(frames_info):
    print(f"Processando frame {i+1}/{len(frames_info)} - ID: {frame_id} tipo_frame_id: {type(frame_id)}")

    # Carrega o frame
    frame = load_frame_from_db(frame_id)
    if frame is None:
        print(f"Erro ao carregar frame ID {frame_id}")
        continue

    print(f"Frame carregado - Tipo: {type(frame)}, Shape: {getattr(frame, 'shape', 'N/A')}")


    # Nomes dos arquivos
    base_name = f"{vid_name}_frame_{frame_num}_t{timestamp:.1f}s"

    # 1. Plota histograma original
    hist_path = f"{output_histogram_analysis}/{base_name}_histogram.png"

    plot_histogram(frame, f"Frame {frame_num} - {timestamp:.1f}s", hist_path)

    # 2. Aplica equalização
    if use_clahe:
        equalized_frame = apply_clahe(frame)
        eq_type = "CLAHE"
    else:
        equalized_frame = equalize_histogram(frame)
        eq_type = "Standard"

        # 3. Salva imagem original e equalizada
        orig_path = f"{output_histogram_analysis}/{base_name}_original.jpg"
        eq_path = f"{output_histogram_analysis}/{base_name}_equalized_{eq_type.lower()}.jpg"

        cv2.imwrite(str(orig_path), frame)
        cv2.imwrite(str(eq_path), equalized_frame)

        # 4. Compara histogramas
        comp_path = f"{output_histogram_analysis}/{base_name}_comparison_{eq_type.lower()}.png"
        compare_histograms(frame, equalized_frame,
                              f"Frame {frame_num} - {eq_type} Equalization", comp_path)

print(f"\nProcessamento concluído! Resultados salvos em: {output_histogram_analysis}")

# Classificação com Machine Learning MODELO NÃO SUPERVISIONADO DE CLUSTERS

###### Extração e visualização dos dados

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
query = f"SELECT * FROM video_frames"
df = pd.read_sql_query(query, conn)
conn.close()


In [ ]:
print(f"Shape dos dados: {df.shape}")
print(f"Colunas: {df.columns.tolist()}")
print(f"\nTipos de dados:\n{df.dtypes.value_counts()}")
print(f"\nValores nulos:\n{df.isnull().sum().sum()}")

##### Extrair dados da coluna metadata_features

In [ ]:
metadata_features = []
for idx, row in df.iterrows():
    try:
        metadata = json.loads(row['custom_metadata'])
        metadata_features.append({
            'brightness_value': metadata.get('brightness_value', 0),
            'frame_std': metadata.get('frame_std', 0),
            'motion_level_meta': metadata.get('motion_level', 'unknown')
        })
    except:
        metadata_features.append({
            'brightness_value': 0,
            'frame_std': 0,
            'motion_level_meta': 'unknown'
        })

In [ ]:
metadata_df = pd.DataFrame(metadata_features)
df = pd.concat([df, metadata_df], axis=1)

In [ ]:
# Criar features adicionais
df['color_range'] = df['std_r'] + df['std_g'] + df['std_b']
df['mean_brightness'] = (df['mean_r'] + df['mean_g'] + df['mean_b']) / 3
df['color_variance'] = (df['std_r'] ** 2 + df['std_g'] ** 2 + df['std_b'] ** 2) / 3

# Features de dominância de cor
df['red_dominance'] = df['mean_r'] / (df['mean_r'] + df['mean_g'] + df['mean_b'] + 1e-6)
df['green_dominance'] = df['mean_g'] / (df['mean_r'] + df['mean_g'] + df['mean_b'] + 1e-6)
df['blue_dominance'] = df['mean_b'] / (df['mean_r'] + df['mean_g'] + df['mean_b'] + 1e-6)

# Converter RGB para HSV aproximado
df['max_rgb'] = df[['mean_r', 'mean_g', 'mean_b']].max(axis=1)
df['min_rgb'] = df[['mean_r', 'mean_g', 'mean_b']].min(axis=1)
df['chroma'] = df['max_rgb'] - df['min_rgb']

# Saturação aproximada
df['saturation'] = np.where(df['max_rgb'] > 0, df['chroma'] / df['max_rgb'], 0)

# Matiz aproximada (simplificada)
df['hue_component'] = np.where(df['chroma'] > 0,
                               np.where(df['max_rgb'] == df['mean_r'],
                                        (df['mean_g'] - df['mean_b']) / df['chroma'],
                                        np.where(df['max_rgb'] == df['mean_g'],
                                                 2 + (df['mean_b'] - df['mean_r']) / df['chroma'],
                                                 4 + (df['mean_r'] - df['mean_g']) / df['chroma'])),
                               0)

##### Preparar features para clustering


In [ ]:
# Features para clustering
feature_cols = [
    # Média das cores
    'mean_r', 'mean_g', 'mean_b',
    # Desvios padrão (vermelho, verde e azul)
    'std_r', 'std_g', 'std_b',
    # Features derivadas
    'brightness_value', 'frame_std',
    'color_range', 'mean_brightness', 'color_variance',
    'red_dominance', 'green_dominance', 'blue_dominance',
    'saturation', 'hue_component'
]

In [ ]:
feature_cols = [col for col in feature_cols if col in df.columns]
for i, col in enumerate(feature_cols):
    print(f"  {i + 1}. {col}")


In [ ]:
# Selecionando o X das features
X = df[feature_cols].values

# normalizar com StandartScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


##### Analisar correlações


In [ ]:
corr_matrix = np.corrcoef(X_scaled.T)

In [ ]:
# mapa de calor
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix,
            xticklabels=feature_cols,
            yticklabels=feature_cols,
            cmap='coolwarm',
            center=0,
            square=True,
            annot=True,
            fmt='.2f',
            cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação entre Features')
plt.tight_layout()
plt.show()

In [ ]:
# Identificar features altamente correlacionadas
high_corr_pairs = []
for i in range(len(corr_matrix)):
    for j in range(i + 1, len(corr_matrix)):
        if abs(corr_matrix[i, j]) > 0.9:
            high_corr_pairs.append((feature_cols[i], feature_cols[j], corr_matrix[i, j]))

In [ ]:
if high_corr_pairs:
    print("\nPares de features altamente correlacionadas (>0.9):")
    for f1, f2, corr in high_corr_pairs:
        print(f"  {f1} <-> {f2}: {corr:.3f}")

#### Determinar numero ótimo de clusters

In [ ]:
max_k = 8
K = range(2, min(max_k + 1, len(X_scaled) // 10))

In [ ]:
inertias = []
silhouettes = []
calinskis = []
davies = []

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)

    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_scaled, labels))
    calinskis.append(calinski_harabasz_score(X_scaled, labels))
    davies.append(davies_bouldin_score(X_scaled, labels))

In [ ]:
# Plotar métricas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Elbow
axes[0, 0].plot(K, inertias, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Número de Clusters (k)')
axes[0, 0].set_ylabel('Inércia')
axes[0, 0].set_title('Método do Cotovelo')
axes[0, 0].grid(True, alpha=0.3)

# Silhouette
axes[0, 1].plot(K, silhouettes, 'go-', linewidth=2, markersize=8)
axes[0, 1].set_xlabel('Número de Clusters (k)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Score (↑ melhor)')
axes[0, 1].grid(True, alpha=0.3)

# Calinski-Harabasz
axes[1, 0].plot(K, calinskis, 'ro-', linewidth=2, markersize=8)
axes[1, 0].set_xlabel('Número de Clusters (k)')
axes[1, 0].set_ylabel('Calinski-Harabasz Score')
axes[1, 0].set_title('Calinski-Harabasz Score (↑ melhor)')
axes[1, 0].grid(True, alpha=0.3)

# Davies-Bouldin
axes[1, 1].plot(K, davies, 'mo-', linewidth=2, markersize=8)
axes[1, 1].set_xlabel('Número de Clusters (k)')
axes[1, 1].set_ylabel('Davies-Bouldin Score')
axes[1, 1].set_title('Davies-Bouldin Score (↓ melhor)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Sugestão baseada em Cotovelo
best_k = K[np.argmax(inertias)]
print(f"\nSugestão baseada em Cotovelo Score: k = {best_k}")

# Sugestão baseada em Silhouette
best_k = K[np.argmax(silhouettes)]
print(f"\nSugestão baseada em Silhouette Score: k = {best_k}")

# Sugestão baseada em calinskis
best_k = K[np.argmax(calinskis)]
print(f"\nSugestão baseada em calinskis Score: k = {best_k}")

# Sugestão baseada em davies
best_k = K[np.argmax(davies)]
print(f"\nSugestão baseada em davies Score: k = {best_k}")

#### Aplicando K-Means para clusterização com calinskis

In [ ]:
n_clusters_silhouette = 2
n_clusters_calinskis = 4
n_clusters_real = 3

kmeans = KMeans(n_clusters=n_clusters_calinskis, random_state=42, n_init=20)
labels = kmeans.fit_predict(X_scaled)

In [ ]:
print(f"\nResultados do K-means com k={n_clusters_calinskis}:")
unique, counts = np.unique(labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count} frames ({count / len(labels) * 100:.1f}%)")

In [ ]:
# Métricas
print(f"\nMétricas de qualidade:")
print(f"  Silhouette Score: {silhouette_score(X_scaled, labels):.3f}")
print(f"  Calinski-Harabasz: {calinski_harabasz_score(X_scaled, labels):.1f}")
print(f"  Davies-Bouldin: {davies_bouldin_score(X_scaled, labels):.3f}")

In [ ]:
def visualizar_clusters_2d(X_scaled, labels, method='pca'):
    """Visualiza clusters em 2D"""

    if method == 'pca':
        reducer = PCA(n_components=2)
        X_2d = reducer.fit_transform(X_scaled)
        var_explained = reducer.explained_variance_ratio_
        title = f'Clusters - PCA (Var: {var_explained[0]:.1%} + {var_explained[1]:.1%})'
    else:  # tsne
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
        X_2d = reducer.fit_transform(X_scaled)
        title = 'Clusters - t-SNE'

    plt.figure(figsize=(10, 8))

    # Plot points
    unique_labels = np.unique(labels)
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))

    for i, label in enumerate(unique_labels):
        mask = labels == label
        plt.scatter(X_2d[mask, 0], X_2d[mask, 1],
                    c=[colors[i]],
                    label=f'Cluster {label}',
                    alpha=0.6,
                    s=50)

    plt.title(title)
    plt.xlabel('Componente 1')
    plt.ylabel('Componente 2')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
visualizar_clusters_2d(X_scaled, labels, 'pca')


In [ ]:
visualizar_clusters_2d(X_scaled, labels, 'tsne')

##### Analise das caracteristicas do cluster

In [ ]:
def analisar_caracteristicas_clusters(df, labels, feature_cols):
    """Analisa características detalhadas de cada cluster"""

    df['cluster'] = labels

    # Estatísticas por cluster
    cluster_stats = df.groupby('cluster')[feature_cols].agg(['mean', 'std'])

    # Para cada cluster, identificar características distintivas
    print("\n=== ANÁLISE DETALHADA DOS CLUSTERS ===")

    for cluster in sorted(df['cluster'].unique()):
        print(f"\n--- CLUSTER {cluster} ---")
        cluster_data = df[df['cluster'] == cluster]
        print(f"Total de frames: {len(cluster_data)} ({len(cluster_data) / len(df) * 100:.1f}%)")

        # Características de cor
        print("\nCaracterísticas de Cor:")
        print(f"  RGB médio: R={cluster_data['mean_r'].mean():.1f}, "
              f"G={cluster_data['mean_g'].mean():.1f}, "
              f"B={cluster_data['mean_b'].mean():.1f}")
        print(f"  Brilho médio: {cluster_data['mean_brightness'].mean():.1f}")
        print(f"  Saturação média: {cluster_data['saturation'].mean():.3f}")

        # Variabilidade
        print(f"\nVariabilidade:")
        print(f"  Desvio padrão de cor (soma): {cluster_data['color_range'].mean():.1f}")
        print(f"  Frame STD médio: {cluster_data['frame_std'].mean():.1f}")

        # Dominância de cor
        print(f"\nDominância de Cor:")
        print(f"  Vermelho: {cluster_data['red_dominance'].mean():.3f}")
        print(f"  Verde: {cluster_data['green_dominance'].mean():.3f}")
        print(f"  Azul: {cluster_data['blue_dominance'].mean():.3f}")

        # Frames exemplo
        print(f"\nFrames exemplo: {cluster_data['frame_number'].head(5).tolist()}")

    # Heatmap das médias
    plt.figure(figsize=(12, 8))
    cluster_means = df.groupby('cluster')[feature_cols].mean()

    # Normalizar para visualização
    cluster_means_norm = (cluster_means - cluster_means.min()) / (cluster_means.max() - cluster_means.min())

    sns.heatmap(cluster_means_norm.T,
                annot=True,
                fmt='.2f',
                cmap='YlOrRd',
                xticklabels=[f'Cluster {i}' for i in cluster_means.index],
                yticklabels=feature_cols,
                cbar_kws={"shrink": 0.8})
    plt.title('Características Normalizadas por Cluster')
    plt.tight_layout()
    plt.show()

    return cluster_stats

In [ ]:
cluster_stats = analisar_caracteristicas_clusters(df, labels, feature_cols)

In [ ]:
# cluster 0 é o love_letter
# cluster 1 é a caneca
# cluster 2 é a banana

# cluster 3 é sombra projetada do meu corpo


In [ ]:
cluster_to_object = {
    0: 'banana',
    1: 'caneca',
    2: 'love_letter',
    3: 'sombra'  # será ignorado na classificação
}

# estou adicionando a coluna 'object_real' para demonstrar os labels reais
df['object_label'] = df['cluster'].map(cluster_to_object)

In [ ]:
# removendo os frames de sombra para evitar ter que treinar eles junto e não confundir o modelo
df_objects = df[df['object_label'] != 'sombra'].copy()

In [ ]:
print(df_objects['object_label'].value_counts())

##### Funções auxiliares para extrair features adicionas para alguns frames de exemplo

In [ ]:
def extract_texture_features(frame):
    """Extrai features de textura usando filtros"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) if len(frame.shape) == 3 else frame

    # Gradientes de Sobel
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    # Magnitude do gradiente
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

    # Features de textura
    texture_features = {
        'gradient_mean': np.mean(gradient_magnitude),
        'gradient_std': np.std(gradient_magnitude),
        'gradient_max': np.max(gradient_magnitude),
        'edge_density': np.sum(gradient_magnitude > np.mean(gradient_magnitude)) / gradient_magnitude.size
    }

    # Laplacian para detectar bordas
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    texture_features['laplacian_var'] = laplacian.var()

    return texture_features

In [ ]:
def extract_shape_features(frame):
    """Extrai features básicas de forma"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) if len(frame.shape) == 3 else frame

    # Threshold adaptativo
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    shape_features = {
        'num_contours': len(contours),
        'total_contour_area': 0,
        'max_contour_area': 0,
        'contour_density': 0
    }

    if contours:
        areas = [cv2.contourArea(c) for c in contours]
        shape_features['total_contour_area'] = sum(areas)
        shape_features['max_contour_area'] = max(areas)
        shape_features['contour_density'] = shape_features['total_contour_area'] / (gray.shape[0] * gray.shape[1])

    return shape_features

In [ ]:
# Extração de frames aleatorios
sample_indices = np.random.choice(df_objects.index, size=min(50, len(df_objects)), replace=False)
texture_features_list = []
shape_features_list = []

for idx in sample_indices:
    frame_id = df_objects.loc[idx, 'id']
    frame = load_frame_from_db(frame_id)

    if frame is not None:
        texture_feat = extract_texture_features(frame)
        shape_feat = extract_shape_features(frame)

        texture_features_list.append({**{'id': frame_id}, **texture_feat})
        shape_features_list.append({**{'id': frame_id}, **shape_feat})

In [ ]:
if texture_features_list:
    texture_df = pd.DataFrame(texture_features_list)
    shape_df = pd.DataFrame(shape_features_list)

    # Merge com o dataframe principal
    df_enhanced = df_objects.merge(texture_df, on='id', how='left')
    df_enhanced = df_enhanced.merge(shape_df, on='id', how='left')

    # Preencher NaN com médias
    for col in texture_df.columns[1:]:
        df_enhanced[col].fillna(df_enhanced[col].mean(), inplace=True)
    for col in shape_df.columns[1:]:
        df_enhanced[col].fillna(df_enhanced[col].mean(), inplace=True)
else:
    df_enhanced = df_objects.copy()

#### Preparação das features para treinar modelo

In [ ]:
# Features originais
feature_cols_ml = [
    'mean_r', 'mean_g', 'mean_b',
    'std_r', 'std_g', 'std_b',
    'brightness_value', 'frame_std',
    'color_range', 'mean_brightness', 'color_variance',
    'red_dominance', 'green_dominance', 'blue_dominance',
    'saturation', 'hue_component'
]

In [ ]:
if 'gradient_mean' in df_enhanced.columns:
    feature_cols_ml.extend(['gradient_mean', 'gradient_std', 'laplacian_var', 'edge_density'])
if 'num_contours' in df_enhanced.columns:
    feature_cols_ml.extend(['num_contours', 'contour_density'])

In [ ]:
available_features = [col for col in feature_cols_ml if col in df_enhanced.columns]
print(f"\nFeatures disponíveis para ML: {len(available_features)}")

#### Separação de X (features) e Y (alvo)

In [ ]:
X = df_enhanced[available_features].values
y = df_enhanced['object_label'].values

In [ ]:
# Dividindo X, Y treino e X, Y teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Normalizar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(f"\nTamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

#### Random Forest

In [ ]:
print("\n=== TREINANDO RANDOM FOREST ===")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)

# Predições
y_pred_rf = rf_model.predict(X_test_scaled)
print("\nRelatório de Classificação - Random Forest:")
print(classification_report(y_test, y_pred_rf))

#### SVM (Suport Vector Machine)

In [ ]:
print("\n=== TREINANDO SVM ===")
svm_model = SVC(kernel='rbf', C=1.0, random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Predições
y_pred_svm = svm_model.predict(X_test_scaled)
print("\nRelatório de Classificação - SVM:")
print(classification_report(y_test, y_pred_svm))

#### Função suporte matrix de confusão

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title, model_name):
    """Plota matriz de confusão detalhada"""
    cm = confusion_matrix(y_true, y_pred)
    classes = sorted(np.unique(y_true))

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title(f'Matriz de Confusão - {title}')
    plt.ylabel('Classe Real')
    plt.xlabel('Classe Predita')

    # Adicionar métricas
    accuracy = np.trace(cm) / np.sum(cm)
    plt.text(0.5, -0.15, f'Acurácia Global: {accuracy:.2%}',
             transform=plt.gca().transAxes, ha='center')

    plt.tight_layout()
    plt.show()

##### Matriz de confusão

In [ ]:
# Random Forest
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest", "RF")

In [ ]:
# SVM
plot_confusion_matrix(y_test, y_pred_svm, "SVM", "SVM")

#### Análise Importância das features

In [ ]:
## Importância das features no Random Forest
feature_importance = pd.DataFrame({
    'feature': available_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'][:15], feature_importance['importance'][:15])
plt.xlabel('Importância')
plt.title('Top 15 Features Mais Importantes - Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
print("\nTop 10 features mais importantes:")
print(feature_importance.head(10))

#### Detecção de objetos

##### Funções auxiliar

In [ ]:
best_model = rf_model  # Acabei escolhendo Random Forest por ter tido resultados similares ao SVM
model_name = "Random Forest"

# Selecionar frames de teste (um de cada objeto)
test_frames = []
for obj in ['love_letter', 'caneca', 'banana']:
    obj_frames = df_enhanced[df_enhanced['object_label'] == obj]
    if len(obj_frames) > 0:
        # Pegar frame do meio do vídeo para cada objeto
        mid_idx = len(obj_frames) // 2
        test_frames.append(obj_frames.iloc[mid_idx])

print(f"\nTestando detecção em {len(test_frames)} frames...")

In [ ]:
def extract_frame_features(frame):
    """
    Extrai features do frame inteiro (como você fazia no clustering)
    """
    # Calcular estatísticas de cor
    if len(frame.shape) == 3:
        b, g, r = cv2.split(frame)
        stats = {
            'mean_r': float(np.mean(r)),
            'mean_g': float(np.mean(g)),
            'mean_b': float(np.mean(b)),
            'std_r': float(np.std(r)),
            'std_g': float(np.std(g)),
            'std_b': float(np.std(b))
        }
    else:
        mean_val = float(np.mean(frame))
        std_val = float(np.std(frame))
        stats = {
            'mean_r': mean_val, 'mean_g': mean_val, 'mean_b': mean_val,
            'std_r': std_val, 'std_g': std_val, 'std_b': std_val
        }

    # Brilho e outras features
    brightness = np.mean(frame)
    frame_std = np.std(frame)

    # Features derivadas (igual ao seu código original)
    color_range = stats['std_r'] + stats['std_g'] + stats['std_b']
    mean_brightness = (stats['mean_r'] + stats['mean_g'] + stats['mean_b']) / 3
    color_variance = (stats['std_r'] ** 2 + stats['std_g'] ** 2 + stats['std_b'] ** 2) / 3

    total = stats['mean_r'] + stats['mean_g'] + stats['mean_b'] + 1e-6
    red_dominance = stats['mean_r'] / total
    green_dominance = stats['mean_g'] / total
    blue_dominance = stats['mean_b'] / total

    max_rgb = max(stats['mean_r'], stats['mean_g'], stats['mean_b'])
    min_rgb = min(stats['mean_r'], stats['mean_g'], stats['mean_b'])
    chroma = max_rgb - min_rgb
    saturation = chroma / max_rgb if max_rgb > 0 else 0

    if chroma > 0:
        if max_rgb == stats['mean_r']:
            hue_component = (stats['mean_g'] - stats['mean_b']) / chroma
        elif max_rgb == stats['mean_g']:
            hue_component = 2 + (stats['mean_b'] - stats['mean_r']) / chroma
        else:
            hue_component = 4 + (stats['mean_r'] - stats['mean_g']) / chroma
    else:
        hue_component = 0

    features = [
        stats['mean_r'], stats['mean_g'], stats['mean_b'],
        stats['std_r'], stats['std_g'], stats['std_b'],
        brightness, frame_std,
        color_range, mean_brightness, color_variance,
        red_dominance, green_dominance, blue_dominance,
        saturation, hue_component
    ]

    return features

In [ ]:
def visualize_detections(obj, frame, detections, title="Detecção de Objetos", output_path=None, show_plot=True):
    """
    Visualiza frame com bounding boxes e labels

    Args:
        frame: imagem em formato BGR (OpenCV)
        detections: lista de detecções
        title: título para visualização
        output_path: caminho para salvar a imagem (opcional)
        show_plot: se deve mostrar o plot com matplotlib
    """
    frame_vis = frame.copy()

    # Cores para cada classe
    colors = {
        'love_letter': (0, 0, 255),  # Vermelho
        'caneca': (255, 255, 255),  # Branco em BGR
        'banana': (0, 255, 255),  # Amarelo em BGR
        'sombra': (128, 128, 128)  # Cinza
    }

    # Desenhar todas as detecções
    for det in detections:
        x, y, w, h = det['bbox']
        label = det['label']
        confidence = det['confidence']

        # Desenhar bounding box
        color = colors.get(label, (255, 255, 255))
        cv2.rectangle(frame_vis, (x, y), (x + w, y + h), color, 2)

        # Adicionar label
        text = f"{label} ({confidence:.2f})"
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]

        # Background para texto
        cv2.rectangle(frame_vis, (x, y - 25), (x + text_size[0], y), color, -1)

        # Texto
        cv2.putText(frame_vis, text, (x, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Salvar imagem se path fornecido
    if output_path is not None:
        try:
            # Criar diretório se não existir
            output_dir = os.path.dirname(f"{output_path}/{obj}")
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir, exist_ok=True)
                print(f"Diretório criado: {output_dir}")

            # Verificar se o frame está no formato correto
            if frame_vis.dtype != np.uint8:
                print(f"Aviso: Convertendo tipo de dados de {frame_vis.dtype} para uint8")
                frame_vis = frame_vis.astype(np.uint8)

            # Salvar imagem
            success = cv2.imwrite(output_path, frame_vis)

            if success:
                print(f"✓ Imagem salva com sucesso: {output_path}")
                # Verificar se o arquivo foi realmente criado
                if os.path.exists(output_path):
                    file_size = os.path.getsize(output_path)
                    print(f"  Tamanho do arquivo: {file_size} bytes")
                else:
                    print("  AVISO: cv2.imwrite retornou True mas arquivo não encontrado!")
            else:
                print(f"✗ Erro ao salvar imagem: {output_path}")
                print(f"  Verifique se o formato é suportado (.jpg, .png, .bmp)")

        except Exception as e:
            print(f"✗ Exceção ao salvar imagem: {e}")
            import traceback
            traceback.print_exc()

    # Mostrar plot se solicitado
    if show_plot:
        plt.figure(figsize=(12, 8))
        # Converter BGR para RGB para matplotlib
        plt.imshow(cv2.cvtColor(frame_vis, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis('off')
        plt.show()

    return frame_vis


In [ ]:
def find_largest_object_bbox(frame, min_area_ratio=0.01, max_area_ratio=0.9):
    """
    Função para encontrar objetos
    """
    height, width = frame.shape[:2]
    total_area = height * width

    print(f"Frame shape: {height}x{width}, total_area: {total_area}")

    # Converter para escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) if len(frame.shape) == 3 else frame
    print(f"Gray shape: {gray.shape}")

    # Blur para reduzir ruído
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # TESTAR DIFERENTES TIPOS DE THRESHOLD
    print("\n=== TESTANDO DIFERENTES THRESHOLDS ===")

    # 1. Threshold adaptativo
    thresh_adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 21, 10
    )

    # 2. Threshold simples (Otsu)
    _, thresh_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # 3. Threshold manual
    _, thresh_manual = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV)

    # Testar cada threshold
    thresholds = {
        'adaptive': thresh_adaptive,
        'otsu': thresh_otsu,
        'manual': thresh_manual
    }

    best_result = None
    best_contour_count = 0

    for thresh_name, thresh in thresholds.items():
        print(f"\n--- Testando {thresh_name} ---")

        # Operações morfológicas
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)

        # Encontrar contornos
        contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print(f"Contornos encontrados: {len(contours)}")

        if not contours:
            continue

        # Analisar contornos
        valid_contours = []
        for i, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            area_ratio = area / total_area

            print(f"  Contorno {i}: área={area}, ratio={area_ratio:.4f}")

            if min_area_ratio <= area_ratio <= max_area_ratio:
                valid_contours.append((contour, area))
                print(f"    ✓ VÁLIDO!")
            else:
                print(f"    ✗ Rejeitado (fora do range {min_area_ratio}-{max_area_ratio})")

        print(f"Contornos válidos: {len(valid_contours)}")

        if valid_contours and len(valid_contours) > best_contour_count:
            best_contour_count = len(valid_contours)
            largest_contour = max(valid_contours, key=lambda x: x[1])[0]
            x, y, w, h = cv2.boundingRect(largest_contour)

            best_result = {
                'bbox': (x, y, w, h),
                'method': thresh_name,
                'contour': largest_contour,
                'thresh_image': cleaned
            }

            print(f"    → Melhor resultado até agora: bbox=({x}, {y}, {w}, {h})")

    return best_result


In [ ]:
def classify_and_detect_single_object(frame, model, scaler, available_features):
    """
    Função de classificar e detectar objetos
    """

    # 1. CLASSIFICAR O FRAME INTEIRO
    frame_features = extract_frame_features(frame)

    # Ajustar features para o modelo
    if len(frame_features) < len(available_features):
        frame_features.extend([0] * (len(available_features) - len(frame_features)))
    frame_features = frame_features[:len(available_features)]


    # Verificar se há NaN ou inf
    if any(np.isnan(frame_features)) or any(np.isinf(frame_features)):
        print("AVISO: Features inválidas detectadas!")
        return []

    # Predizer classe do frame
    frame_features_scaled = scaler.transform([frame_features])
    predicted_class = model.predict(frame_features_scaled)[0]

    if hasattr(model, 'predict_proba'):
        proba = model.predict_proba(frame_features_scaled)[0]
        confidence = max(proba)
        print(f"Probabilidades: {dict(zip(model.classes_, proba))}")
    else:
        confidence = 1.0

    print(f"Frame classificado como: {predicted_class} (confiança: {confidence:.2f})")

    result = find_largest_object_bbox(frame)

    if result is None:
        print("Nenhum objeto encontrado!")
        return []

    print(f"Objeto encontrado! Método: {result['method']}, BBox: {result['bbox']}")

    return [{
        'bbox': result['bbox'],
        'label': predicted_class,
        'confidence': confidence,
        'area': result['bbox'][2] * result['bbox'][3],  # w * h
        'detection_method': result['method']
    }]


In [ ]:
def detect_single_frame(frame_id, obj):
    """
    Detecção para unico frame por vez
    """
    frame = load_frame_from_db(frame_id)
    if frame is None:
        print("Frame não encontrado!")
        return

    detections = classify_and_detect_single_object(frame, best_model, scaler, available_features)

    # Salvar frame original para inspeção
    cv2.imwrite(f"data/bounding_box_images/{obj}/{frame_id}_original.jpg", frame)
    print(f"Frame original salvo: data/bounding_box_images/{obj}/{frame_id}_original.jpg")

    if detections:
        print("\nDETECÇÃO BEM-SUCEDIDA!")
        for i, det in enumerate(detections):
            print(f"Detecção {i+1}:")
            print(f"  Label: {det['label']}")
            print(f"  Confiança: {det['confidence']:.2f}")
            print(f"  BBox: {det['bbox']}")
            print(f"  Método: {det.get('detection_method', 'N/A')}")

        # Visualizar resultado
        vis_frame = visualize_detections(obj, frame, detections, f"Debug Frame {frame_id}", show_plot=False)
        cv2.imwrite(f"data/bounding_box_images/{obj}/{frame_id}_detected.jpg", vis_frame)
        print(f"Resultado salvo: data/bounding_box_images/{obj}/{frame_id}_detected.jpg")
    else:
        print("\nNENHUMA DETECÇÃO!")

    return detections

In [ ]:
# Pegar um frame de cada classe para testar
test_frames_debug = []
for obj in ['banana', 'love_letter', 'caneca']:
    obj_frames = df_enhanced[df_enhanced['object_label'] == obj]
    if len(obj_frames) > 0:
        for i in range(min(10, len(obj_frames))):
            frame_info = obj_frames.iloc[i]
            print(f"\nTestando {obj} - Frame ID: {frame_info['id']}")
            detections = detect_single_frame(int(frame_info['id']), obj)
            test_frames_debug.append((obj, frame_info, detections))

print(f"\n=== RESUMO DA EXECUÇÃO ===")
for obj, frame_info, detections in test_frames_debug:
    status = "OK" if detections else "FALHOU"
    print(f"{obj}: {status}")

In [ ]:
incorrect_predictions = X_test_scaled[y_test != y_pred_rf]
incorrect_true = y_test[y_test != y_pred_rf]
incorrect_pred = y_pred_rf[y_test != y_pred_rf]

In [ ]:
if len(incorrect_predictions) > 0:
    print(f"\nTotal de erros: {len(incorrect_predictions)} ({len(incorrect_predictions) / len(y_test) * 100:.1f}%)")

    # Analisar padrões de erro
    error_patterns = pd.DataFrame({
        'true_label': incorrect_true,
        'predicted_label': incorrect_pred
    })

    print("\nPadrões de erro mais comuns:")
    print(error_patterns.groupby(['true_label', 'predicted_label']).size().sort_values(ascending=False))

    # Análise de features dos erros
    print("\n\nAnálise de features médias por tipo de erro:")
    for true_label in np.unique(incorrect_true):
        mask = incorrect_true == true_label
        if np.sum(mask) > 0:
            print(f"\n{true_label} classificado incorretamente:")
            error_features = incorrect_predictions[mask].mean(axis=0)

            # Comparar com features médias corretas
            correct_mask = (y_train == true_label)
            if np.sum(correct_mask) > 0:
                correct_features = X_train_scaled[correct_mask].mean(axis=0)

                # Maiores diferenças
                diff = np.abs(error_features - correct_features)
                top_diff_idx = np.argsort(diff)[-5:][::-1]

                print("  Features com maior diferença:")
                for idx in top_diff_idx:
                    print(f"    {available_features[idx]}: {diff[idx]:.3f}")

In [ ]:
print("\n=== CONCLUSÃO ===")
print(f"Melhor modelo: {model_name}")
print(f"Acurácia no teste: {np.mean(y_test == y_pred_rf):.2%}")
print("\nO modelo está pronto para classificar os objetos no vídeo!")

In [ ]:
import joblib

# Salvar o melhor modelo e o scaler
joblib.dump(best_model, 'models/best_model.pkl')
joblib.dump(scaler, 'models/scaler.pkl')
print("\nModelo e scaler salvos em 'best_model.pkl' e 'scaler.pkl'")

# Transcrição de Aúdio

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
from pathlib import Path

# Obtém o caminho absoluto da pasta 'ffmpeg' no seu projeto
project_root = Path.cwd()  # Pasta onde o Jupyter está executando
ffmpeg_path = project_root / "ffmpeg"

os.environ["PATH"] += os.pathsep + str(ffmpeg_path)

In [ ]:
# Comando especifico para macOS
#
# ! xattr -d com.apple.quarantine ffmpeg/ffmpeg
# ! xattr -d com.apple.quarantine ffmpeg/ffprobe

# OU USAR ESTE:
# ! chmod +x ffmpeg/ffmpeg
# ! chmod +x ffmpeg/ffprobe

In [ ]:
# Caminho do arquivo MP3
audio_mp3 = "data/audio.mp3"

# Converte MP3 para WAV
audio = AudioSegment.from_mp3(audio_mp3)

In [ ]:
audio_wav = "temp_audio.wav"
audio.export(audio_wav, format="wav")

In [ ]:
# Inicializa o reconhecedor
r = sr.Recognizer()

# Lê o arquivo de áudio
with sr.AudioFile(audio_wav) as source:
    audio_data = r.record(source)

# Transcreve o áudio para texto
try:
    texto = r.recognize_google(audio_data, language="pt-BR")
    print("Texto transcrito:")
    print(texto)
except sr.UnknownValueError:
    print("Não foi possível entender o áudio.")
except sr.RequestError as e:
    print(f"Erro ao se conectar ao serviço de reconhecimento: {e}")

In [ ]:
# Salva em um arquivo .txt
with open("data/transcricao_audio/transcricao.txt", "w", encoding="utf-8") as arquivo:
    arquivo.write(texto)

In [ ]:
if os.path.exists("temp_audio.wav"):
    os.remove("temp_audio.wav")
    print("Arquivo 'temp_audio.wav' deletado com sucesso!")
else:
    print("Arquivo 'temp_audio.wav' não encontrado.")

In [ ]:
!pip freeze